In [1]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv('PlannedCondosWPB - Sheet1.csv',dtype='str')

## Data clean

In [3]:
df['Name '] = df['Name '].str.replace('Unnamed','',regex=True)
df = df.rename(columns={'Name ':'Name'})

## Geocode

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
df['Site Address'] = df['Site Address'] + ' West Palm Beach, FL'

In [6]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Site Address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Link Formatter

In [8]:
df['Story Link'] = 'Click here for story'

df["short_description"] = df["Story Link"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Story Link'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["Story Link"] = '<a href="' + df["Link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]

In [12]:
def tooltip_html(row_index, max_chars=30):
    """Generate HTML content for the tooltip with line breaks for long text."""
    tooltip_content = '<div class="popup-content">'

    # Dynamically add data for each column
    for column in df.columns:
        value = df.at[row_index, column]
        value_str = str(value).strip()
        if pd.notnull(value) and value_str != "" and column in ['Developer','Name ','Site Address','Units','Stories','Est. Completion']:

            # Check if the string exceeds the maximum character length
            if len(value_str) > max_chars and column == "Site Address":
                # Insert a <br> tag every max_chars characters
                value_str = '<br>'.join(value_str[i:i+max_chars] for i in range(0, len(value_str), max_chars))

            tooltip_content += f'<strong>{column}:</strong> {value_str}<br>'

    tooltip_content += '</div>'
    return tooltip_content

# Initialize the map
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=13,tiles=None)

# Define the URL template for Jawg Maps tiles
jawg_url = 'https://tile.jawg.io/jawg-streets/{z}/{x}/{y}{r}.png?access-token=Kr5DL29m5aQey4XgiDKLupcLHZyD0EZvbmORtosm3iyO47oCJavHcmt09qPe42hz'

# Add the Jawg Maps tile layer to the map
folium.TileLayer(
    tiles=jawg_url,  # URL template
    attr='Map data &copy; <a href="https://www.jawg.io" target="_blank">Jawg Maps</a>',  # Attribution (required by Jawg Maps)
    name='Jawg Streets',  # Name of the tile layer, as it will appear in layer controls
    overlay=False,  # Whether this layer is an overlay (False means it's a base map layer)
    control=True  # Whether to include this layer in LayerControl widgets
).add_to(m)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format("Mapping Residential Developments in West Palm Beach")

m.get_root().html.add_child(folium.Element(title_html))

# Loop through the DataFrame to create markers
for i in range(len(df)):
    # Create tooltip HTML content
    tooltip_content = tooltip_html(i)
    
    # Create a custom icon
    icon = folium.Icon(color='red')
    
    # Create a marker with a tooltip
    marker = folium.CircleMarker(
        location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
        tooltip=tooltip_content,
        color='red',
        fill=True,
        radius=7,
        icon=folium.CircleMarker(color='red')
    ).add_to(m)

# Display the map
m

In [10]:
m.save('index.html')

In [11]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/WPB_Branded_Condos_04_02_24
